In [ ]:
from sys import version_info
import os
import io
import tarfile
import pandas as pd
import numpy as np
import joblib
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import mlflow
import dvc.api
import warnings

warnings.filterwarnings("ignore")

PYTHON_VERSION = "{major}.{minor}.{micro}".format(major=version_info.major,
                                                  minor=version_info.minor,
                                                  micro=version_info.micro)
pipeline_path = "pipe.pkl"
model_path = "model.pkl"

path = 'dataset/housing.csv'
repo = '../.git'
version = 'v1' #Git tag

data_url = dvc.api.get_url(
    path = path,
    repo = repo,
    rev = version
)

In [ ]:
def fetch_data_from_s3(url):
    data = dvc.api.read(
            path = path,
            repo = repo,
            rev = version
        )
    data = io.StringIO(data)
    return pd.read_csv(data, sep=',')


def fetch_data_from_fs(url):
    return pd.read_csv(url, sep=',')


def fetch_data(url):
    storage_type = url.split(":")[0]
    if storage_type.upper() == "S3":
        data = fetch_data_from_s3(url.split(":")[-1])
    else:
        data = fetch_data_from_fs(url)
        
    return data

In [ ]:
backend_uri = os.environ['MLFLOW_TRACKING_URI']
artifact_uri = os.environ['MLFLOW_ARTIFACT_STORE']
mlflow.set_tracking_uri(backend_uri)

In [ ]:
housing = fetch_data(data_url)
housing.head()

### Create pipeline

In [ ]:
targetCol = "median_house_value"
catCols = ["ocean_proximity"]
numCols = ["housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income"]

num_pipeline = Pipeline([
                    ("Imputer", SimpleImputer()),
                    ("Scaler", StandardScaler())
                ])

full_pipeline = ColumnTransformer([
                    ("Numerical_Pipeline", num_pipeline, numCols),
                    ("OneHot", OneHotEncoder(), catCols)
                ])

### Data processing

In [ ]:
x = housing[numCols + catCols]
y = housing[targetCol]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True)
x_train = full_pipeline.fit_transform(x_train)
x_test = full_pipeline.transform(x_test)

### Build Model

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))

### Save the pipeline and the model

In [ ]:
joblib.dump(model, model_path)
joblib.dump(full_pipeline, pipeline_path);

In [ ]:
# Create an `artifacts` dictionary that assigns a unique name to the saved pipeline and the model
# This dictionary will be passed to `mlflow.pyfunc.save_model`, which will copy the model file
# into the new MLflow Model's directory.
artifacts = {
    "pipeline": pipeline_path,
    "model": model_path
}

# Define the model class
class ModelWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        self.pipeline = joblib.load(context.artifacts["pipeline"])
        self.model = joblib.load(context.artifacts["model"])

    def predict(self, context, model_input):
        input_matrix = self.pipeline.transform(model_input)
        return self.model.predict(input_matrix)

In [ ]:
# Create a Conda environment for the new MLflow Model that contains all necessary dependencies.
import cloudpickle
conda_env = {
    'channels': ['defaults'],
    'dependencies': [
      'python={}'.format(PYTHON_VERSION),
      'pip',
      {
        'pip': [
          'mlflow',
          'scikit-learn=={}'.format(sklearn.__version__),
          'joblib=={}'.format(joblib.__version__),
          'cloudpickle=={}'.format(cloudpickle.__version__),
        ],
      },
    ],
    'name': 'model_env'
}

In [ ]:
experiment = mlflow.get_experiment_by_name("simple_pipeline")
if experiment is None:
    experiment_id = mlflow.create_experiment("simple_pipeline", artifact_location=artifact_uri)
    experiment = mlflow.get_experiment(experiment_id)

mlflow.set_experiment(experiment.name)

mlflow_pyfunc_model_path = "mlflow_simple_pipeline"
with mlflow.start_run():
    mlflow.log_param('data_url', data_url)
    mlflow.log_param('data_version', version)
    mlflow.log_param('input_rows', x_train.shape[0])
    mlflow.log_param('input_columns', x_train.shape[1])
    
    mlflow.log_metric("rmse", rmse)
    mlflow.pyfunc.log_model(mlflow_pyfunc_model_path, 
                            python_model=ModelWrapper(), 
                            artifacts=artifacts,
                            conda_env=conda_env)

In [ ]:
# Save the MLflow Model
# mlflow.pyfunc.save_model(
#         path=mlflow_pyfunc_model_path, python_model=ModelWrapper(), artifacts=artifacts,
#         conda_env=conda_env)